In [1]:
import time
import pickle
import pandas as pd
import numpy as np
import math

# internal tools
import dsgtools.azure

In [3]:
path = "Analytics/Personal Folders/liuwei01/2023/JPMC/"
jpmc = pd.read_adls(path+ "0307_armbs_1803_jpmc_attributes_w20230302-084435.csv", reader = pd.read_csv, encoding='iso-8859-1', 
                    usecols = ["accountnumber", "historydateyyyymm", "seleid", "errorcode", "time_ms", "bs035_bal_m0_fin"], low_memory=False)
jpmc.columns = [x + "_jpmc" for x in jpmc.columns]
sbfe = pd.read_adls(path+ "0307_armbs_1803_legacy_sbfe_attributes_w20230302-073425.csv", reader = pd.read_csv, encoding='iso-8859-1', 
                    usecols = ["accountnumber", "historydateyyyymm", "seleid", "errorcode", "time_ms", "sbfehitindex", "sbfeaccountcount", "sbfeopencount"], low_memory=False)
print(jpmc.shape)
print(sbfe.shape)

(147171, 6)
(147176, 8)


In [4]:
merged = jpmc.merge(sbfe, left_on = "accountnumber_jpmc", right_on = "accountnumber")
print(merged.shape)

(147171, 14)


In [40]:
merged["bs035_bal_m0_fin_jpmcflag"] = np.where(merged["bs035_bal_m0_fin_jpmc"] >=999999990, merged["bs035_bal_m0_fin_jpmc"], 1)

In [41]:
print(merged.sbfehitindex.value_counts(dropna = False))
print(merged.sbfeaccountcount.value_counts(dropna = False))
print(merged.bs035_bal_m0_fin_jpmcflag.value_counts(dropna = False))

1.0    111826
0.0     35343
NaN         2
Name: sbfehitindex, dtype: int64
-99.0    147169
 NaN          2
Name: sbfeaccountcount, dtype: int64
1.0            63333
999999990.0    57563
999999997.0    15020
999999996.0     8048
999999999.0     3148
999999994.0       59
Name: bs035_bal_m0_fin_jpmcflag, dtype: int64


In [10]:
merged["time_ms"].min()

0

In [33]:
merged["valid_time"] = np.where((merged["time_ms_jpmc"] > 0) & (merged["time_ms"] > 0), 1, 0)
merged["diff_ms"] = np.where(merged["valid_time"]  == 1, merged["time_ms_jpmc"] - merged["time_ms"], None)
merged["diff_ms"] = merged["diff_ms"].astype(float)
merged["jpmc_slower"] = np.where(merged["valid_time"]  == 0, None, 
                                 np.where(merged["diff_ms"] > 0, 1, 0))
print(merged["jpmc_slower"].value_counts(dropna = False))
print("JPMC attribute return slower on ", round(merged["jpmc_slower"].mean(skipna = True) *100, 2) , "% of records")

0.0    87459
1.0    59710
NaN        2
Name: jpmc_slower, dtype: int64
JPMC attribute return slower on  40.57 % of records


In [35]:
report = merged[['time_ms_jpmc', 'time_ms', 'diff_ms']].describe(include = 'all')
report.loc['null count'] = merged.isnull().sum()
report.loc['% null count'] = merged.isnull().mean()
report.round(1)

,time_ms_jpmc,time_ms,diff_ms
count,147171.0,147171.0,147169.0
mean,1678.6,1712.1,-33.6
std,4129.3,3980.5,5557.1
min,0.0,0.0,-172741.0
25%,891.0,981.0,-251.0
50%,1079.0,1141.0,-65.0
75%,1429.0,1430.0,147.0
max,189455.0,174002.0,168984.0
null count,0.0,0.0,2.0
% null count,0.0,0.0,0.0


In [ ]:
temp = merged[merged["jpmc_slower"] == 1]
report = temp[[, 'time_ms_jpmc', 'time_ms', 'diff_ms']].describe([0.01, 0.05, 0.1, .25, .5, .75, 0.9, 0.95, 0.99], include = 'all')
report.loc['null count'] = merged.isnull().sum()
report.loc['% null count'] = merged.isnull().mean()
report.round(1)